In [1]:
# run-DSS.R
# Runs Dispersion Shrinkage Estimation method on methylation data (one chromosome at a time)
library(data.table)
library(tidyverse)
library(argparse)
library(DSS)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB

Filter first!!!

In [2]:
parser <- ArgumentParser()
parser$add_argument("--chr", default= "chr22", help='Chromosome to run DSS on')
parser$add_argument("--covariates", default= "NK,CD4T")
parser$add_argument("--num_pcs", default= "2", help = 'number of principal components (precomputed)')
args <- parser$parse_known_args()

# Setup output directory, will save called regions, figures?
odir <- paste0("result-", args[[1]]$chr)
dir.create(odir, showWarning = FALSE)

In [3]:
pc.df <- read_table( file.path("../../data/prin_comps/", paste0("PC_", args[[1]]$chr , ".tsv")) )


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  PC1 = col_double(),
  PC2 = col_double(),
  PC3 = col_double(),
  PC4 = col_double(),
  PC5 = col_double(),
  PC6 = col_double(),
  PC7 = col_double(),
  PC8 = col_double(),
  PC9 = col_double(),
  PC10 = col_double(),
  sample = col_double(),
  num_null = col_double(),
  num_not_null = col_double(),
  mean_methylation = col_double(),
  mean_coverage = col_double(),
  median_coverage = col_double()
)




In [4]:

ss.df <- read_csv("../../data/meta/phenos-cleaned.csv")
DT <- fread(file.path("../../data/cov-meth/", paste0(args[[1]]$chr, ".tsv")))

Rows: 83 Columns: 18

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): cohort, machine, pool, group, batch, sex
dbl (12): sample, CD8T, CD4T, NK, Bcell, Mono, Gran, charlson_score, bmi, wa...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



In [5]:
M <- dcast(DT, pos~sample, value.var="methylated")
Cov <- dcast(DT, pos~sample, value.var="coverage")

In [6]:
valid_samples <- intersect(intersect(colnames(M), ss.df$sample), pc.df$sample)

filt.df <- ss.df %>%
            inner_join(pc.df, by = "sample") %>% 
            dplyr::filter(sample %in% valid_samples) 

# pack years smoking needs to be not na
filt.df$pack_years[is.na(filt.df$pack_years)] <- 0

drop_cols <- as.character(setdiff(colnames(M), valid_samples))[-1] # dont remove pos
M[, (drop_cols):=NULL]
Cov[, (drop_cols):=NULL]



In [7]:
length(valid_samples)

[1] 61

In [8]:
a <- c(1,2,3, NULL)
a

[1] 1 2 3

In [9]:

threshold <- 0
drop.ix <- ( rowSums(is.na(M)) > threshold)
#M <- M[!drop.ix, ]
#Cov <- Cov[!drop.ix, ]
#table(drop.ix)
M[is.na(M)] <- 0
Cov[is.na(Cov)] <- 0

In [10]:
# Order needs to be correct!!!
M <- M[ , c("pos", filt.df$sample), with = FALSE]
Cov <- Cov[ , c("pos", filt.df$sample), with = FALSE]

In [11]:
# create bs seq object, needs chromosome identiifer, methylated reads, and unmethylated reads
bs <- BSseq(chr = rep(DT$chr[1], nrow(M)), pos = M$pos,
            M = as.matrix(M[ , -c("pos"), with=FALSE]), 
            Cov = as.matrix(Cov[, -c("pos"), with=FALSE]), 
            sampleNames = names(M)[-1])


In [12]:
# Print this check in the future!!!!
all( filt.df$sample == colnames(bs) )

[1] TRUE

In [13]:
names(filt.df)

[1] "sample"           "cohort"           "machine"          "pool"            
 [5] "group"            "batch"            "CD8T"             "CD4T"            
 [9] "NK"               "Bcell"            "Mono"             "Gran"            
[13] "charlson_score"   "bmi"              "waist_circumf"    "age"             
[17] "sex"              "pack_years"       "PC1"              "PC2"             
[21] "PC3"              "PC4"              "PC5"              "PC6"             
[25] "PC7"              "PC8"              "PC9"              "PC10"            
[29] "num_null"         "num_not_null"     "mean_methylation" "mean_coverage"   
[33] "median_coverage"

In [14]:
#TODO: formula from input flags
dml.fit <- DMLfit.multiFactor(bs, design = filt.df, smoothing = TRUE, smoothing.span = 100, 
        formula = ~cohort + machine + CD8T + CD4T + NK + Bcell + Mono + bmi + age + sex)

Fitting DML model for CpG site: 100000 , 200000 , 300000 , 400000 , 500000 , 600000 , 700000 , 800000 , 900000 , 1000000 , 1100000 , 1200000 , 1300000 , 1400000 , 1500000 , 1600000 , 

In [15]:
colnames(dml.fit$X)
test.cohort <- DMLtest.multiFactor(dml.fit, coef = 2)

[1] "(Intercept)"     "cohortCONTROL"   "machineNovaseq2" "CD8T"           
 [5] "CD4T"            "NK"              "Bcell"           "Mono"           
 [9] "bmi"             "age"             "sexM"

In [16]:
table(is.na(test.cohort$stat))


  FALSE    TRUE 
1486006  125237 

In [17]:
methylation <- M / Cov
methylation$pos <- M$pos
fwrite(methylation, file.path(odir, "methylation.csv"))

In [18]:
save(list = c("test.cohort"), file= file.path(odir, "test-values.RData"))
fwrite(M, file.path(odir, "M.csv"))
fwrite(Cov, file.path(odir, "Cov.csv"))


In [19]:
dmrs <- callDMR(test.cohort, p.threshold=0.01, minCG = 5 )

In [20]:
dmrs

,chr,start,end,length,nCG,areaStat
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
649,chr6,10659742,10660199,458,38,-117.62179
7287,chr6,149450767,149450862,96,20,56.93582
555,chr6,9024345,9024516,172,17,47.34443
1630,chr6,28802606,28802846,241,8,27.85109
8226,chr6,166005956,166006031,76,9,-26.15201
6709,chr6,137342911,137343152,242,8,-25.42136
2271,chr6,38796656,38796813,158,8,24.93634
8354,chr6,168216790,168216914,125,9,24.50036
8449,chr6,169425536,169425634,99,8,-23.07389


In [21]:
X <- model.matrix(~ cohort + machine + CD8T + CD4T + NK + Bcell + Mono + bmi + age + sex, filt.df)

In [22]:
round( rowSums(filt.df[, c("NK", "Mono", "CD4T", "CD8T", "Bcell", "Gran")]), 2)

[1] 1.01 1.00 1.00 1.00 0.99 1.01 1.00 1.00 1.01 1.00 1.01 1.00 1.00 1.01 1.00
[16] 1.00 1.00 1.00 1.00 0.99 1.00 1.00 1.01 0.99 1.00 0.99 0.99 1.00 0.99 0.99
[31] 1.01 0.99 0.99 1.06 1.00 1.01 0.99 1.00 1.01 1.00 1.00 1.01 0.99 0.99 0.99
[46] 1.00 0.99 1.02 0.99 1.00 1.00 1.00 1.06 0.99 1.01 0.99 1.00 1.01 0.99 1.00
[61] 1.04

In [23]:
colSums(filt.df[, c("NK", "Mono", "CD4T", "CD8T", "Bcell", "Gran")])

NK      Mono      CD4T      CD8T     Bcell      Gran 
 3.855671  4.452582  9.063332  1.719980  2.567237 39.451656

In [24]:
cor(X)

Warning message in cor(X):
“the standard deviation is zero”


,(Intercept),cohortCONTROL,machineNovaseq2,CD8T,CD4T,NK,Bcell,Mono,bmi,age,sexM
(Intercept),1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
cohortCONTROL,NA,1.000000000,0.58086980,0.047861323,0.15708097,0.17918023,0.079074487,0.003505138,0.231553857,0.205490003,-0.05627706
machineNovaseq2,NA,0.580869805,1.00000000,0.176007007,0.02539451,0.26291593,0.195327104,0.048454004,0.201852785,0.374177094,-0.03004499
CD8T,NA,0.047861323,0.17600701,1.000000000,-0.15102879,0.41196357,0.086307586,-0.043545616,0.007386018,0.001697975,-0.03314351
CD4T,NA,0.157080972,0.02539451,-0.151028786,1.00000000,-0.08309296,0.068016489,-0.277860282,-0.148387486,-0.059137822,-0.08879252
NK,NA,0.179180227,0.26291593,0.411963572,-0.08309296,1.00000000,0.059258466,-0.033818399,-0.022756261,0.335654853,-0.02750347
Bcell,NA,0.079074487,0.19532710,0.086307586,0.06801649,0.05925847,1.000000000,0.007516909,0.085229462,0.104474243,-0.03842846
Mono,NA,0.003505138,0.04845400,-0.043545616,-0.27786028,-0.03381840,0.007516909,1.000000000,-0.100649893,0.146028811,0.21888798
bmi,NA,0.231553857,0.20185279,0.007386018,-0.14838749,-0.02275626,0.085229462,-0.100649893,1.000000000,0.007670984,-0.12690616
age,NA,0.205490003,0.37417709,0.001697975,-0.05913782,0.33565485,0.104474243,0.146028811,0.007670984,1.000000000,-0.10155286
